In [7]:
import ImageRepresentation
from rsift_extraction import _GetSingleImageFeature
import numpy as np
import cv2 as cv
import os
from delf import feature_io
%matplotlib inline
from matplotlib import pyplot as plt 
from matplotlib import image as img
query_dir = "/mnt/datasets/daejeon_sv_v1_query_delf/"
query_images_dir = "/mnt/datasets/daejeon_sv_v1_query_full/"
database_dir = "/mnt/datasets/daejeon_sv_v1_vlad256_delf/"
image_dir = "/mnt/datasets/daejeon_sv_v1/"
saved_model = "/mnt/datasets/codebook/delf_256_model.pkl"

In [2]:
database = []
indexes = []
for f in os.listdir(database_dir):
    filename = f.split('.')
    vector = np.load(os.path.join(database_dir, f), allow_pickle=True)
    indexes.append(filename[0])
    database.append(vector)

In [26]:
# Train the pca model on the database
from sklearn.decomposition import IncrementalPCA
n_components = 2048
transformer = IncrementalPCA(n_components=n_components, whiten=True, batch_size=n_components)
reduced_database = transformer.fit_transform(database)
import joblib
pca_model = "/mnt/datasets/pca/delf_vlad256_" + str(n_components) + "_whiten.pkl"
joblib.dump(transformer, pca_model)

/home/canhld/.local/lib/python3.5/site-packages/sklearn/decomposition/incremental_pca.py:294: RuntimeWarning: Mean of empty slice.
  explained_variance[self.n_components_:].mean()
/home/canhld/.local/lib/python3.5/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['/mnt/datasets/pca/delf_vlad256_2048_whiten.pkl']

In [4]:
# L2 norm again the database
pca_database = []
for vector in reduced_database:
    l2_norm = np.linalg.norm(vector)
    normed_vector = vector/l2_norm
    pca_database.append(normed_vector)

In [16]:
# Load the query set
ir = ImageRepresentation.ImageRepresentation(name = 'delf_vlad', saved_model = saved_model )
print(ir.codebook.cluster_centers_.shape)
pca_model = "/mnt/datasets/pca/delf_vlad256_whiten.pkl"
query_vectors = []
queries = []
query_images = []
for f in os.listdir(query_dir):
    filename, ext = f.split('.')
    if ext == 'DELF' or ext == 'delf':
        queries.append(filename)
        im = cv.imread(os.path.join(query_images_dir, filename + '.jpg'), flags=cv.IMREAD_UNCHANGED)
        query_images.append(cv.cvtColor(im, cv.COLOR_RGB2BGR))
        _,_,features,_,_ = feature_io.ReadFromFile(os.path.join(query_dir, f))
        vector = ir.VLADEncode(image_features=features, apply_pca=True, n_components=n_components, pca_model=pca_model)
        query_vectors.append(vector)

(256, 40)


In [17]:
predicts = []
top_ranks = 50
for query,image,query_vector in zip(queries, query_images, query_vectors):
    rank_list = []
    predict = []
    predict.append(query)
    for vector, index in zip(pca_database,indexes):
        rank_list.append((index, np.dot(vector, query_vector)))
    rank_list.sort(key = lambda tup: tup[1], reverse=True)
    for i in range(top_ranks):
        predict.append(rank_list[i][0])
    predicts.append(predict)
print(predicts)

[['IMG_0420', '1878_gsv_2', '2310_gsv_4', '1963_gsv_2', '1877_gsv_2', '2051_gsv_1', '3176_gsv_3', '2310_gsv_2', '2137_gsv_2', '2309_gsv_4', '447_gsv_7', '2310_gsv_5', '968_gsv_3', '2051_gsv_2', '1207_gsv_4', '1879_gsv_2', '2308_gsv_4', '3002_gsv_7', '1791_gsv_2', '872_gsv_3', '3175_gsv_5', '7_gsv_0', '2225_gsv_5', '1207_gsv_5', '3263_gsv_4', '2309_gsv_5', '361_gsv_7', '2224_gsv_4', '3349_gsv_3', '2050_gsv_1', '887_gsv_7', '1305_gsv_7', '1119_gsv_2', '1877_gsv_1', '279_gsv_6', '45_gsv_4', '1967_gsv_0', '15_gsv_7', '3090_gsv_3', '1878_gsv_1', '13_gsv_3', '1878_gsv_3', '3175_gsv_4', '355_gsv_1', '3002_gsv_0', '700_gsv_3', '528_gsv_2', '134_gsv_6', '2250_gsv_4', '3090_gsv_4', '22_gsv_7'], ['IMG_0588', '3353_gsv_6', '2827_gsv_2', '1026_gsv_0', '948_gsv_4', '2484_gsv_7', '3092_gsv_5', '3265_gsv_6', '2830_gsv_2', '890_gsv_2', '2483_gsv_6', '1881_gsv_5', '522_gsv_6', '2996_gsv_7', '2827_gsv_7', '3092_gsv_3', '2310_gsv_7', '1967_gsv_2', '3169_gsv_4', '15_gsv_6', '3153_gsv_4', '17_gsv_0', '3179_

In [18]:
ref_file = "/mnt/datasets/daejeon_sv_v1_query_50m.csv"
with open(ref_file) as f:
    contents = f.readlines()
lines = [x.strip('\n') for x in contents]
ref = dict()
for line in lines:
    tup = line.split(',')
    ref[tup[0]] = set(tup[1:])
results = []
images = []
for predict in predicts:
    result = []
    query = predict[0]
#     result.append(query)
    rank_list = predict[1:]
    first_correct = 0
    for item in rank_list:
        if item[:-6] in ref[query]:
            result.append(1)
        else:
            result.append(0)
    results.append(np.array(result))

for query,result in zip(queries,results):
    print(query, result)
# Recall calculation:
recall_results = []
for result in results:
    recall_result = []
    first_correct = 0
    for item in result:
        if item == 1:
            for i in range(first_correct, len(result)):
                recall_result.append(1)
            break
        else:
            recall_result.append(0)
            first_correct += 1
    recall_results.append(np.array(recall_result))

top_retrieval = 20
recall = np.array([0]*top_retrieval)
for recall_result in recall_results:
    recall += recall_result[:top_retrieval]
recall = recall/float(len(queries))
print(recall)

# Precision calculation
precision = []
precision_rank = np.array([0]*top_retrieval)
for result in results:
    precision_rank += result[:top_retrieval]
s = 0
for i in range(0,len(precision_rank)):
    s += precision_rank[i]
    precision.append(s/((i+1)*len(queries)))
print(np.array(precision))

IMG_0420 [1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 0]
IMG_0588 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0]
IMG_0415 [1 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0]
IMG_0419 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 0]
IMG_0398 [1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0]
IMG_0413 [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
IMG_0405 [0 0 0 1 1 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0]
IMG_0394 [0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 0 0 0 0 0 0 0]
IMG_0402 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0

In [20]:
print(transformer.get_params())

{'whiten': True, 'batch_size': 512, 'n_components': 512, 'copy': True}


['/mnt/datasets/pca/delf_vlad256_whiten.pkl']